In [2]:
import pandas as pd
import numpy as np

In [23]:
file_name ="dancing_20250204-170238.csv"
df_raw =pd.read_csv(f"/app/yolo_structure/results/{file_name}")
# df_raw["datetime"] = pd.to_datetime(df_raw["datetime"], format="%Y%m%d-%H%M%S.%f")
df_raw .tail(5)             

,datetime,distance,event,detected,source_name,frame_hash,frame_num
2996,20250204-170307,0,PERSON,person,dancing.mp4,dancing_PERSON_3,443
2997,20250204-170307,0,PERSON,person,dancing.mp4,dancing_PERSON_4,443
2998,20250204-170307,0,PERSON,person,dancing.mp4,dancing_PERSON_6,443
2999,20250204-170307,0,PERSON,person,dancing.mp4,dancing_PERSON_7,443
3000,20250204-170307,0,PERSON,person,dancing.mp4,dancing_PERSON_8,443


In [24]:
# frame_hash별로 그룹핑하고, 1초 이내 5개 이상 존재하는 경우 찾기
df_raw_person = df_raw[df_raw["detected"] == "person"]
df_raw_person

# frame_hash별로 그룹화 후 frame_num 연속성 확인
df_filtered_person = pd.DataFrame()

grouped = df_raw_person.groupby("frame_hash")
for _, group in grouped:
    group = group.sort_values(by="frame_num").reset_index(drop=True)
    group["diff"] = group["frame_num"].diff().fillna(1)
    group["consec_group"] = (group["diff"] != 1).cumsum()
    valid_groups = group.groupby("consec_group").filter(lambda x: len(x) >= 5)
    df_filtered_person = pd.concat([df_filtered_person, valid_groups.drop(columns=["diff", "consec_group"])])

# 결과 출력
df_filtered_person

,datetime,distance,event,detected,source_name,frame_hash,frame_num
0,20250204-170239,0,PERSON,person,dancing.mp4,dancing_PERSON_1,1
1,20250204-170239,0,PERSON,person,dancing.mp4,dancing_PERSON_1,2
2,20250204-170239,0,PERSON,person,dancing.mp4,dancing_PERSON_1,3
3,20250204-170239,0,PERSON,person,dancing.mp4,dancing_PERSON_1,4
4,20250204-170242,0,PERSON,person,dancing.mp4,dancing_PERSON_1,5
...,...,...,...,...,...,...,...
145,20250204-170303,0,PERSON,person,dancing.mp4,dancing_PERSON_9,231
146,20250204-170303,0,PERSON,person,dancing.mp4,dancing_PERSON_9,232
147,20250204-170303,0,PERSON,person,dancing.mp4,dancing_PERSON_9,233
148,20250204-170303,0,PERSON,person,dancing.mp4,dancing_PERSON_9,234


In [ ]:
llm_res_l = []
for index,value in df_filtered_person.iterrows():
    if value["detected"] !="person":
        llm_res_l.append(value["detected"])
        continue
    try:
        crop_filename = os.path.join(crop_save_dir,value["frame_hash"]+"_"+ str(value["frame_num"]) +".jpg")
        res = ollama.chat(
            model= "llama3.2-vision:11b",
            messages=[
                {
                    'role': 'user',
                    'content': "tell me what is the person doing",
                    'images': [crop_filename]
                }
            ]
        )
        
        print(res['message']['content'])
    except:
       continue

    final_result = res['message']['content'].upper().replace(".","").replace(" ","")
    print(f"{crop_filename} / Ollama Result: {final_result}")
    llm_res_l.append(final_result)
df_filtered_person["llm_res"] = llm_res_l
df_filtered_person

The individual depicted in the image appears to be engaged in a shopping activity, as evidenced by their presence within a grocery store and the items visible in the background.
/app/yolo_structure/results/frame_img/dancing_PERSON_1_1.jpg / Ollama Result: THEINDIVIDUALDEPICTEDINTHEIMAGEAPPEARSTOBEENGAGEDINASHOPPINGACTIVITY,ASEVIDENCEDBYTHEIRPRESENCEWITHINAGROCERYSTOREANDTHEITEMSVISIBLEINTHEBACKGROUND
The individual appears to be walking through a shopping center or other commercial area. Their posture and hands suggest they are either looking at something specific, perhaps an item for sale or a display, or may have stopped to read a sign, map, or advertisement. The context of their surroundings suggests they might be on a casual outing, running errands, or simply enjoying the environment.
/app/yolo_structure/results/frame_img/dancing_PERSON_1_2.jpg / Ollama Result: THEINDIVIDUALAPPEARSTOBEWALKINGTHROUGHASHOPPINGCENTEROROTHERCOMMERCIALAREATHEIRPOSTUREANDHANDSSUGGESTTHEYAREEITHERLOOKINGA

In [7]:
# frame_hash별로 그룹핑하고, 1초 이내 5개 이상 존재하는 경우 찾기
df_raw_stop = df_raw[df_raw["event"] == "STOP"]
df_raw_stop

# frame_hash별로 그룹화 후 frame_num 연속성 확인
df_filtered_stop = pd.DataFrame()

grouped = df_raw_stop.groupby("frame_hash")
for _, group in grouped:
    group = group.sort_values(by="frame_num").reset_index(drop=True)
    group["diff"] = group["frame_num"].diff().fillna(1)
    group["consec_group"] = (group["diff"] != 1).cumsum()
    valid_groups = group.groupby("consec_group").filter(lambda x: len(x) >= 5)
    df_filtered_stop = pd.concat([df_filtered_stop, valid_groups.drop(columns=["diff", "consec_group"])])

# 결과 출력
df_filtered_stop

,datetime,distance,event,detected,source_name,frame_hash,frame_num
48,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,466
49,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,467
50,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,468
51,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,469
52,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,470
...,...,...,...,...,...,...,...
144,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1784
145,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1785
146,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1786
147,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1787


In [14]:
# frame_hash별로 그룹핑하고, 1초 이내 5개 이상 존재하는 경우 찾기
df_raw_reverse = df_raw[df_raw["event"] == "REVERSE"]
df_raw_reverse

# frame_hash별로 그룹화 후 frame_num 연속성 확인
df_filtered_reverse = pd.DataFrame()

grouped = df_raw_reverse.groupby("frame_hash")
for _, group in grouped:
    group = group.sort_values(by="frame_num").reset_index(drop=True)
    group["diff"] = group["frame_num"].diff().fillna(1)
    group["consec_group"] = (group["diff"] != 1).cumsum()
    valid_groups = group.groupby("consec_group").filter(lambda x: len(x) >= 3)
    df_filtered_reverse = pd.concat([df_filtered_reverse, valid_groups.drop(columns=["diff", "consec_group"])])

# 결과 출력
df_filtered_reverse

,datetime,distance,event,detected,source_name,frame_hash,frame_num
67,20250204-163610,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_1,507
68,20250204-163610,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_1,508
69,20250204-163610,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_1,509
29,20250204-163606,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,123
30,20250204-163606,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,124
31,20250204-163606,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,125
254,20250204-163635,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,1325
255,20250204-163635,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,1326
256,20250204-163635,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_2,1327
168,20250204-163613,0,REVERSE,car,cctv001.stream,cctv001_REVERSE_3,644


In [15]:
import ollama
import os
import cv2

origin_dir = "/app"
## 최종 저장경로 설정!!
results_dir = os.path.join(origin_dir,"yolo_structure","results")
## 사건파일저장
crop_save_dir = os.path.join(results_dir,"frame_img") 

In [17]:
df_filtered_stop

,datetime,distance,event,detected,source_name,frame_hash,frame_num
48,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,466
49,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,467
50,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,468
51,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,469
52,20250203-165515,0,STOP,car,cctv001.stream,cctv001_STOP_1,470
...,...,...,...,...,...,...,...
144,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1784
145,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1785
146,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1786
147,20250203-165559,0,STOP,car,cctv001.stream,cctv001_STOP_5,1787


In [19]:
 os.path.join(crop_save_dir,value["frame_hash"]+"_"+ str(value["frame_num"]) +".jpg")

'/app/yolo_structure/results/frame_img/cctv001_STOP_1_466.jpg'

In [ ]:
1095/30

In [22]:
llm_res_l = []
for index,value in df_filtered_stop.iterrows():
    if value["detected"] !="car":
        llm_res_l.append(value["detected"])
        continue
    try:
        crop_filename = os.path.join(crop_save_dir,value["frame_hash"]+"_"+ str(value["frame_num"]) +".jpg")
        res = ollama.chat(
            model= "llama3.2-vision:11b",
            messages=[
                {
                    'role': 'user',
                    'content': "tell me what is in this image. Answer only one in [SEDAN/SUV/VAN/TRUCK/BUS] answer in one word",
                    'images': [crop_filename]
                }
            ]
        )
        
        print(res['message']['content'])
    except:
       continue

    final_result = res['message']['content'].upper().replace(".","").replace(" ","")
    print(f"{crop_filename} / Ollama Result: {final_result}")
    llm_res_l.append(final_result)
df_filtered_stop["llm_res"] = llm_res_l
df_filtered_stop

Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_468.jpg / Ollama Result: SEDAN
Van.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_469.jpg / Ollama Result: VAN
Bus.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_470.jpg / Ollama Result: BUS
Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_665.jpg / Ollama Result: SEDAN
Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_666.jpg / Ollama Result: SEDAN
Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_667.jpg / Ollama Result: SEDAN
Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_689.jpg / Ollama Result: SEDAN
Bus.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_737.jpg / Ollama Result: BUS
Truck.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_738.jpg / Ollama Result: TRUCK
Truck.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_740.jpg / Ollama Result: TRUCK
Sedan.
/app/yolo_structure/results/frame_img/cctv001_STOP_1_880.jpg / Ollama Result: SEDAN
Sedan.
/app

ValueError: Length of values (200) does not match length of index (407)

In [ ]:
llm_res_l = []
for index,value in df_filtered_reverse.iterrows():
    if value["detected"] !="car":
        llm_res_l.append(value["detected"])
        continue
   
    crop_filename = os.path.join(crop_save_dir,value["frame_hash"]+"_"+ str(value["frame_num"]) +".jpg")
    res = ollama.chat(
        model= "llama3.2-vision:11b",
        messages=[
            {
                'role': 'user',
                'content': "tell me what is in this image. Answer only one in [SEDAN/SUV/VAN/TRUCK/BUS] answer in one word",
                'images': [crop_filename]
            }
        ]
    )
    
    print(res['message']['content'])

    final_result = res['message']['content'].upper().replace(".","").replace(" ","")
    print(f"{crop_filename} / Ollama Result: {final_result}")
    llm_res_l.append(final_result)
df_filtered_reverse["llm_res"] = llm_res_l
df_filtered_reverse